In [10]:
require 'nngraph'
require 'gnuplot'
require 'nn'

checkpoint_1 = torch.load('../../arxiv/1Conv_tanh_sky.t7')   
a = torch.Tensor(checkpoint_1.train_loss_history) 



In [11]:
gnuplot.pngfigure('plot_sky_class_32_1.png')
gnuplot.plot(a)
gnuplot.plotflush()